In [2]:
import pandas as pd
import numpy as np

In [3]:
n_users = 10000
def gen_data(n_users, binary_treatment=True, with_income=False):
    
    if with_income:
        income = np.random.normal(500, scale=15, size=n_users)
        gender = np.random.randint(0, 2, size=n_users)
        coupon = gender * 20 + 110 + income / 50 + np.random.normal(scale=5, size=n_users)
        if binary_treatment:
            coupon = (coupon > 120).astype(int)
        amount = coupon * 150 + gender * 100 + 150 + income / 5 + np.random.normal(size=n_users)
        time_spent = coupon * 10 + amount / 10

        df = pd.DataFrame({
            'gender': gender,
            'coupon': coupon,
            'amount': amount,
            'income': income,
            'time_spent': time_spent,
        })

        return df
    else:
        gender = np.random.randint(0, 2, size=n_users)
        coupon = gender * 20 + 150 + np.random.normal(scale=5, size=n_users)
        if binary_treatment:
            coupon = (coupon > 150).astype(int)
        amount = coupon * 30 + gender * 100 + 150 + np.random.normal(size=n_users)
        time_spent = coupon * 100 + amount / 10

        df = pd.DataFrame({
            'gender': gender,
            'coupon': coupon,
            'amount': amount,
            'time_spent': time_spent,
        })
        
        return df

df = gen_data(n_users, with_income=True)
df['treatment'] = df['coupon'] > 0
df

,gender,coupon,amount,income,time_spent,treatment
0,1,1,501.153466,502.859522,60.115347,True
1,1,1,498.444645,489.891213,59.844464,True
2,0,1,399.377098,506.497926,49.937710,True
3,1,1,498.180144,488.499506,59.818014,True
4,0,1,402.350945,505.573211,50.235095,True
...,...,...,...,...,...,...
9995,1,1,496.183684,484.916315,59.618368,True
9996,1,1,501.365788,507.337451,60.136579,True
9997,1,1,500.128792,497.786771,60.012879,True
9998,0,0,252.610341,514.541506,25.261034,False
